In [ ]:
%matplotlib

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [ ]:
plt.plot(np.random.randn(50).cumsum())

In [ ]:
import os
myList = []
with os.scandir('avalonlogs/logs') as entries:
    for entry in entries:
        myList.append(pd.json_normalize(json.load(open(entry))))

df = pd.concat(myList)


In [ ]:
df

In [ ]:
df['hasMerlin'] =   df.apply(lambda row: any([True for role in row['outcome.roles'] if role['role'] == "MERLIN"]), axis=1)
df['hasPercival'] = df.apply(lambda row: any([True for role in row['outcome.roles'] if role['role'] == "PERCIVAL"]), axis=1)
df['hasOberon'] = df.apply(lambda row: any([True for role in row['outcome.roles'] if role['role'] == "OBERON"]), axis=1)
df['hasMordred'] = df.apply(lambda row: any([True for role in row['outcome.roles'] if role['role'] == "MORDRED"]), axis=1)
df['hasMorgana'] = df.apply(lambda row: any([True for role in row['outcome.roles'] if role['role'] == "MORGANA"]), axis=1)
df['numPlayers'] = df.apply(lambda row: len(row['outcome.roles']), axis=1)


df['evilWin'] = df.apply(lambda row: row['outcome.state'] == "EVIL_WIN", axis=1)
df['evilWinByAssassination'] = df.apply(lambda row: row['evilWin'] and row['outcome.message'] == "Merlin assassinated", axis=1)

#df[df["outcome.message"] == "Merlin assassinated"]
#df.groupby(['hasMerlin', 'hasPercival', 'hasOberon', 'hasMordred', 'evilWin']).agg(['count'])
summary = df.groupby(['hasMerlin','hasMorgana', 'hasPercival', 'evilWin']).agg(['count'])['missions']
summary['pct'] = summary.groupby(level=[0,1,2]).apply(lambda x: 100 * x / float(x.sum()))
summary

In [ ]:
def roleForName(roles, name):
    for role in roles:
        if (role['name'] == name):
            return role['role']
    return None

In [ ]:
df['roleAssassinated'] = df.apply (lambda row: roleForName(row['outcome.roles'], row['outcome.assassinated']), axis=1)
df

In [ ]:
#assassinStats = (df.groupby(['numPlayers','roleAssassinated'], dropna=False).agg(['count']))['missions']
assassinStats = (df.groupby(['numPlayers', 'hasPercival', 'roleAssassinated']).agg(['count']))['missions']
assassinStats['pct'] = assassinStats.groupby(level=[0, 1]).apply(lambda x: 100 * x / float(x.sum()))
assassinStats

In [ ]:
df[df["outcome.message"] == "Merlin assassinated"]

In [ ]:
assassinStats = (df.groupby(['hasMerlin', 'hasMorgana', 'hasPercival', 'evilWin'], dropna=False).agg(['count']))['missions']
assassinStats['pct'] = assassinStats.groupby(level=[0,1,2]).apply(lambda x: 100 * x / float(x.sum()))
assassinStats


In [ ]:
assassinStats = (df.groupby(['hasMerlin', 'assassinationAttempted'], dropna=False).agg(['count']))['missions']
assassinStats['pct'] = assassinStats.groupby(level=[0]).apply(lambda x: 100 * x / float(x.sum()))
assassinStats
